<div style = text-align:center>
<h2>🤖ADIVINADOR DE ANIMALES🐾🗨</h2>
<h3>Inteligencia Artificial</h3>
<h3>Camilo Delgado, Robert Aguilar, Camilo Oliveros, Lorena Rueda, Jesús Infante</h3>
    <br>
<p>Se requiere un sistema que pueda identificar el animal en que se piensa, basándose en una serie de preguntas. El sistema debe almacenar lo que va aprendiendo en una base de datos, ampliando en cada sondeo su conocimiento.  El sistema debe preguntar usando un sintetizador de voz y esperar una respuesta de audio del usuario que posteriormente debe ser convertida a texto para su procesamiento.</p>
</div>

In [ ]:
pip install pyttsx3

In [ ]:
pip install speech_recognition

In [ ]:
pip install json

In [ ]:
pip install tkinter

In [ ]:
import pyttsx3
import speech_recognition as sr
import json
import tkinter as tk

# Crear objeto de sintetizador
engine = pyttsx3.init()

# Crear objeto de reconocimiento de voz
r = sr.Recognizer()


# Establecer la voz del sintetizador
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)  # 1 corresponde a la voz femenina y 0 a la masculina


# Creando ventana de la interfaz gráfica
window = tk.Tk()
window.title("🐵ADIVINADOR DE ANIMALES🐾")
window.geometry("500x350")
window.configure(bg="#FFFCE6")

# Crear etiquetas y cajas de texto para mostrar la interacción
label = tk.Label(window, text="¡¡¡Hola soy BOT 🤖🖐!!!", font=('Segoe UI Emoji', 10), fg="#2A576B", bg="#FFFCE6")
label.pack()

conversation_label = tk.Label(window, text="Conversación:", font=('Segoe UI Emoji', 10), fg="#2A576B", bg="#FFFCE6")
conversation_label.pack()

conversation_text = tk.Text(window, height=10, width=60)
conversation_text.pack()

response_label = tk.Label(window, text="🤖¡VAMOS A JUGAR!🤖", font=('Segoe UI Emoji', 12), fg="#FFA500", bg="#FFFCE6")
response_label.pack()

user_text = tk.Text(window, height=2, width=60)
user_text.pack()

# Definir una función para el sintetizador de voz
def speak(text):
    engine.say(text)
    engine.runAndWait()


# Definir una función para el reconocimiento de voz
def listen():
    with sr.Microphone() as source:
        print('Escuchando...')
        audio = r.listen(source)
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source)
        r.energy_threshold = 3000
        r.language = 'es-ES'
        try:
            print('Procesando...')
            query = r.recognize_google(audio, language='es-CO')
            print(f'Usuario: {query}')
            conversation_text.insert(tk.END, f'Usuario: {query}\n')
            conversation_text.see(tk.END)
            window.update()
        except Exception as e:
            print(e)
            return 'None'
        return query


name = 'camilo'


def get_name():
    # Saludar al usuario
    speak('Hola, ¿cómo te llamas?')
    conversation_text.insert(tk.END, '🤖Bot: Hola, ¿cómo te llamas?\n')
    conversation_text.see(tk.END)
    window.update()
    name = listen()
    # Responder con un mensaje personalizado
    if name != 'None':
        user_text.delete(1.0, tk.END)
        user_text.insert(tk.END, f'¡Ahora estoy jugando con {name}, adivinaré el animal que piensas!')
        conversation_text.see(tk.END)
        window.update()
        speak(f'Hola {name}, bienvenido al programa.')
    # else:
    # speak('Lo siento, no pude entender tu nombre.')


name_button = tk.Button(window, text="Obtener nombre", command=get_name, bg="#FFA500", fg="white")
name_button.pack()

# -----------------------------------------------------------

class Arbol:
    def __init__(self, carga=None, izq=None, der=None):
        self.carga = carga
        self.izquierda = izq
        self.derecha = der

    def to_dict(self):
        data = {
            'carga': self.carga,
            'izquierda': None,
            'derecha': None
        }

        if self.izquierda:
            data['izquierda'] = self.izquierda.to_dict()

        if self.derecha:
            data['derecha'] = self.derecha.to_dict()

        return data

    def update_json(self, filename):
        data = self.to_dict()

        with open(filename, 'w') as file:
            json.dump(data, file)

    def insertar(self, valor, filename):
        if self.carga is None:
            self.carga = valor
            self.izquierda = Arbol()
            self.derecha = Arbol()
        elif valor < self.carga:
            self.izquierda.insertar(valor, filename)
        elif valor > self.carga:
            self.derecha.insertar(valor, filename)

    @classmethod
    def cargar_desde_json(cls, filename):
        with open(filename, 'r') as file:
            data = json.load(file)

        return cls.construir_desde_dict(data)

    @classmethod
    def construir_desde_dict(cls, data):
        carga = data['carga']
        izquierda_data = data['izquierda']
        derecha_data = data['derecha']

        izquierda = cls.construir_desde_dict(izquierda_data) if izquierda_data else None
        derecha = cls.construir_desde_dict(derecha_data) if derecha_data else None

        return cls(carga, izquierda, derecha)

    def __str__(self):
        return str(self.carga)


def si(preg):
    resp = listen()
    # resp = input(preg)
    return resp[0] == 's'


# -----------------------------------------------------------

def get_respuestas():
    bucle = True
    arbol = None  # Variable para almacenar el árbol
    arbol = Arbol.cargar_desde_json('arbol.json')

    while bucle:
        speak("¿Estás pensando en un animal? ")
        conversation_text.insert(tk.END, '🤖Bot: ¿Estás pensando en un animal?\n')
        conversation_text.see(tk.END)
        window.update()

        if not si("¿Estás pensando en un animal? "):
            break

        arbol_actual = arbol
        while arbol_actual.izquierda:
            speak("¿El animal " + arbol_actual.carga + "? ")
            conversation_text.insert(tk.END, 'Bot: ¿El animal ' + arbol_actual.carga + '?\n')
            conversation_text.see(tk.END)
            window.update()

            if si("¿El animal " + arbol_actual.carga + "? "):
                arbol_actual = arbol_actual.izquierda
            else:
                arbol_actual = arbol_actual.derecha

        animal = arbol_actual.carga
        speak("¿Es un " + animal + "? ")
        conversation_text.insert(tk.END, '🤖Bot: ¿Es un ' + animal + '?\n')
        conversation_text.see(tk.END)
        window.update()

        if si("¿Es un " + animal + "? "):
            speak("¡Soy el más grande!")
            conversation_text.insert(tk.END, '🤖Bot: ¡Soy el más grande!\n')
            conversation_text.see(tk.END)
            window.update()
            continue
        speak("¿Qué animal era? ")
        conversation_text.insert(tk.END, '🤖Bot: ¿Qué animal era?\n')
        conversation_text.see(tk.END)
        window.update()

        nuevo = listen()
        # nuevo = input("¿Qué animal era? ")
        speak("¿Qué diferencia a un " + animal + " de un " + nuevo + "? ")
        conversation_text.insert(tk.END, '🤖Bot: ¿Qué diferencia a un ' + animal + ' de un ' + nuevo + '?\n')
        conversation_text.see(tk.END)
        window.update()

        info = listen()
        # info = input("¿Qué diferencia a un " + animal + " de un " + nuevo + "? ")

        arbol_actual.carga = info
        arbol_actual.derecha = Arbol(animal)
        arbol_actual.izquierda = Arbol(nuevo)

        arbol.update_json('arbol.json')

    speak('Gracias por jugar conmigo')
    conversation_text.insert(tk.END, '🤖Bot: 🙄Gracias por jugar conmigo🙄\n')
    conversation_text.see(tk.END)
    window.update()


def close_window():
    window.destroy()


button2 = tk.Button(window, text="Juguemos 🎮", command=get_respuestas, bg="lightgreen")
button2.pack()

close_button = tk.Button(window, text="Cerrar", command=close_window, bg="red")
close_button.pack()

window.mainloop()



Escuchando...
